In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import gensim

import os
import re

print(np.__version__)
print(pd.__version__)
print(tf.__version__)
print(nltk.__version__)
print(gensim.__version__) # 원래 4.1.2였음

1.21.4
1.3.3
2.6.0
3.6.5
3.8.3


# ChatbotData 다운로드
[ChatbotData](https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv)

- 11,876개의 문답 페어로 이루어진 데이터
- 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링되어 있다

In [2]:
!wget https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv

--2024-01-23 13:32:05--  https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv [following]
--2024-01-23 13:32:05--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData.csv.2’

ChatbotData.csv.2   100%[===================>] 868.99K  --.-KB/s    in 0.04s   

2024-01-23 13:32:05 (20.6 MB/s) - ‘ChatbotData.csv.2’ saved [889842/889842]



In [3]:
ChatbotData = pd.read_csv('./ChatbotData.csv')
ChatbotData.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
questions = ChatbotData['Q']
answers = ChatbotData['A']

# 데이터 정제

In [5]:
def preprocess_sentence(sentence):
    # 영문자 소문자로 변환
    sentence = sentence.lower()
    
    # 정규식을 사용하여 영문자, 한글, 숫자, 주요 특수문자 제외 모든 문자 제거
    sentence = re.sub(r"[^a-zA-Z가-힣0-9.,!?]", " ", sentence)
    
    # 여러 공백을 하나의 공백으로 축소
    sentence = re.sub(r"\s+", " ", sentence)
    
    return sentence.strip()

# 테스트
example_sentence = "Hello, 안녕하세요!      123 This is a test sentence... #preprocessing"
processed_sentence = preprocess_sentence(example_sentence)
print(processed_sentence)

hello, 안녕하세요! 123 this is a test sentence... preprocessing


# 데이터 토큰화

In [6]:
from konlpy.tag import Mecab
mecab = Mecab()


def build_corpus(src_sentences, tgt_sentences, tokenize_fn, threshold=20):
    src_corpus, tgt_corpus = [], []
    seen_pairs = set()

    for src, tgt in zip(src_sentences, tgt_sentences):
        # 정제 및 토큰화
        src = preprocess_sentence(src)
        tgt = preprocess_sentence(tgt)
        src_tokens = tokenize_fn(src)
        tgt_tokens = tokenize_fn(tgt)

        # 일정 길이 이상인 문장 및 중복된 문장 제외
        if len(src_tokens) <= threshold and len(tgt_tokens) <= threshold:
            pair = (src, tgt)
            if pair not in seen_pairs:
                src_corpus.append(src_tokens)
                tgt_corpus.append(tgt_tokens)
                seen_pairs.add(pair)

    return src_corpus, tgt_corpus

# 토큰화에는 mecab.morphs 함수를 사용
que_corpus, ans_corpus = build_corpus(questions, answers, mecab.morphs)

# 결과 출력 (예시로 상위 5개 출력)
print("소스 문장 예시:", que_corpus[:5])
print("타겟 문장 예시:", ans_corpus[:5])

소스 문장 예시: [['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'], ['ppl', '심하', '네']]
타겟 문장 예시: [['하루', '가', '또', '가', '네요', '.'], ['위로', '해', '드립니다', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['눈살', '이', '찌푸려', '지', '죠', '.']]


# Augmentation

In [7]:
!pip install gdown

In [8]:
import gdown

# 다운로드 받을 파일의 Google Drive 공유 링크
google_drive_link = "https://drive.google.com/uc?id=0B0ZXk88koS2KbDhXdWg1Q2RydlU"

# 다운로드 받을 파일의 이름
output_filename = "ko.zip"

# 다운로드 받을 경로
output_path = os.path.join("/aiffel/aiffel/DeepGoing_NLP", output_filename)

# gdown을 사용하여 Google Drive에서 파일 다운로드
gdown.download(google_drive_link, output_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=0B0ZXk88koS2KbDhXdWg1Q2RydlU
From (redirected): https://drive.google.com/uc?id=0B0ZXk88koS2KbDhXdWg1Q2RydlU&confirm=t&uuid=69623ebc-e5f3-4eb4-a425-a50c1265e03e
To: /aiffel/aiffel/DeepGoing_NLP/ko.zip
100%|██████████| 80.6M/80.6M [00:01<00:00, 67.0MB/s]


'/aiffel/aiffel/DeepGoing_NLP/ko.zip'

In [9]:
from zipfile import ZipFile

zip_path = "/aiffel/aiffel/DeepGoing_NLP/ko.zip"
extracted_path = "/aiffel/aiffel/DeepGoing_NLP/"

# Extract the contents of the ZIP file
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

In [10]:
# 라이브러리 호환문제로 에러가 발생하여 다운그레이드를 진행
'''
Can't get attribute 'Vocab' on <module 'gensim.models.word2vec' from '/opt/conda/lib/python3.9/site-packages/gensim/models/word2vec.py'>
'''
!pip install --upgrade gensim==3.8.3

In [11]:
from gensim.models import Word2Vec

# 다운로드 받은 ko.bin 파일의 경로를 적절히 설정
model_path = "./ko.bin"

# Word2Vec 모델 로드
model = Word2Vec.load(model_path)

# 모델 사용 예시
similar_words = model.wv.most_similar('단어', topn=5)
print(similar_words)

[('낱말', 0.8457916975021362), ('용어', 0.7469839453697205), ('어휘', 0.7309367656707764), ('접미사', 0.6993076801300049), ('접두사', 0.6960497498512268)]


In [12]:
import random
from sklearn.metrics.pairwise import cosine_similarity

def lexical_sub(sentence, embedding_model, top_n=5):
    """
    주어진 문장에 대해 단어 임베딩을 기반으로 한 어휘 대체를 수행합니다.

    Args:
    - sentence (str): 어휘 대체를 수행할 입력 문장입니다.
    - embedding_model: 사전 훈련된 단어 임베딩 모델 (예: Word2Vec, FastText).
    - top_n (int, optional): 각 단어를 대체할 상위 후보의 수 (기본값은 5).

    Returns:
    - str: 어휘 대체 후의 증강된 문장입니다.
    """
    words = sentence.split()
    augmented_sentence = []

    for word in words:
        if word in embedding_model:
            word_embedding = np.array([embedding_model[word]])
            word_similarities = embedding_model.similar_by_vector(word_embedding.flatten(), topn=top_n)
            top_candidates = [candidate for candidate, _ in word_similarities]

            # 가장 유사한 후보로 단어를 대체합니다.
            augmented_sentence.append(random.choice(top_candidates))
        else:
            augmented_sentence.append(word)

    return ' '.join(augmented_sentence)

In [13]:
# Augmentation 수행 및 데이터 확장
augmented_que_corpus = []
augmented_ans_corpus = []

# 원본 데이터에 대해 Augmentation 수행
for que_list, ans_list in zip(que_corpus, ans_corpus):
    augmented_que_list = [lexical_sub(que, model.wv) for que in que_list]
    augmented_ans_list = [lexical_sub(ans, model.wv) for ans in ans_list]
    
    augmented_que_corpus.append(augmented_que_list)
    augmented_ans_corpus.append(augmented_ans_list)

# 반대로도 수행하여 데이터 3배로 늘리기
for que_list, ans_list in zip(que_corpus, ans_corpus):
    augmented_que_list = [lexical_sub(ans, model.wv) for ans in ans_list]
    augmented_ans_list = [lexical_sub(que, model.wv) for que in que_list]
    
    augmented_que_corpus.append(augmented_que_list)
    augmented_ans_corpus.append(augmented_ans_list)

# 결과 확인
print("원본 문장:", que_corpus[0])
print("Augmentation된 문장:", augmented_que_corpus[0])

# 데이터의 크기 확인
print("원본 데이터 크기:", len(que_corpus))
print("Augmentation된 데이터 크기:", len(augmented_que_corpus))

원본 문장: ['12', '시', '땡', '!']
Augmentation된 문장: ['12', '시의', '경음', '짱']
원본 데이터 크기: 11599
Augmentation된 데이터 크기: 23198


In [ ]:
import csv

# 파일로 저장할 경로 지정
save_path = "/path/to/save/augmented_data.csv"

# Augmentation된 데이터를 행 단위로 CSV 파일에 저장
with open(save_path, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # 헤더 작성
    csvwriter.writerow(["source", "target"])
    
    # Augmentation된 데이터 작성
    for augmented_que_list, augmented_ans_list in zip(augmented_que_corpus, augmented_ans_corpus):
        csvwriter.writerow([" ".join(augmented_que_list), " ".join(augmented_ans_list)])

print("Augmentation된 데이터가 성공적으로 저장되었습니다.")

# 데이터 벡터화

In [14]:
# 타겟 데이터에 <start> 토큰과 <end> 토큰 추가
ans_corpus_with_tokens = [["<start>"] + ans + ["<end>"] for ans in ans_corpus]

# 결과 확인
print(ans_corpus_with_tokens[0])


['<start>', '하루', '가', '또', '가', '네요', '.', '<end>']


In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 소스 데이터와 타겟 데이터 결합
combined_corpus = que_corpus + ans_corpus_with_tokens  # ans_corpus_with_tokens는 <start>, <end> 토큰 추가된 데이터

# 토크나이저 생성 및 단어 사전 구축
tokenizer = Tokenizer(filters='', oov_token='<OOV>')
tokenizer.fit_on_texts(combined_corpus)

# 단어 사전 크기 확인
vocab_size = len(tokenizer.word_index) + 1
print("단어 사전 크기:", vocab_size)

# 소스 데이터와 타겟 데이터를 정수 시퀀스로 변환
enc_train = tokenizer.texts_to_sequences(que_corpus)
dec_train = tokenizer.texts_to_sequences(ans_corpus_with_tokens)

# 패딩 작업
enc_train = pad_sequences(enc_train, padding='post')
dec_train = pad_sequences(dec_train, padding='post')

# 결과 확인
print("소스 데이터 (enc_train) 크기:", enc_train.shape)
print("타겟 데이터 (dec_train) 크기:", dec_train.shape)


단어 사전 크기: 6785
소스 데이터 (enc_train) 크기: (11599, 20)
타겟 데이터 (dec_train) 크기: (11599, 22)


# 훈련하기

In [17]:
def positional_encoding(pos, d_model):
    # 각 포지션과 차원에 대한 각도를 계산하는 함수
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    # 각 포지션에 대한 각도 벡터를 생성하는 함수
    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    # 각 포지션과 차원에 대한 사인과 코사인 값을 가지는 테이블을 생성
    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    # 짝수 차원에는 사인 값을, 홀수 차원에는 코사인 값을 적용
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # 짝수 차원
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # 홀수 차원

    return sinusoid_table

In [18]:
def generate_padding_mask(seq):
    # 입력 시퀀스에서 패딩(0)에 해당하는 위치를 찾아 마스크 생성
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    # 룩어헤드 마스크 생성: 현재 위치 이후의 위치에 대해 마스킹
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    # 인코더 입력 패딩 마스크 생성
    enc_mask = generate_padding_mask(src)

    # 디코더-인코더 어텐션 마스크 생성 (인코더 입력의 패딩 위치 마스킹)
    dec_enc_mask = generate_padding_mask(src)

    # 디코더 마스크 생성 (패딩 위치와 룩어헤드 마스킹을 포함)
    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [19]:
import tensorflow as tf

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        # 각각의 선형 변환을 수행하는 레이어들을 정의
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        # 마지막에 사용되는 선형 변환 레이어
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        # 스케일링된 점곱 어텐션을 수행하는 메소드
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        # 마스크 적용 (필요한 경우)
        if mask is not None:
            scaled_qk += (mask * -1e9)  

        # 소프트맥스 함수를 통해 어텐션 가중치 계산
        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        # 가중 평균을 통해 출력 계산
        out = tf.matmul(attentions, V)

        return out, attentions
        
    def split_heads(self, x):
        # 주어진 텐서를 여러 헤드로 나누는 메소드
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        # 나누어진 헤드를 결합하는 메소드
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x
    
    def call(self, Q, K, V, mask):
        # 주어진 Q, K, V에 대한 멀티헤드 어텐션을 수행하는 메소드
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        # 각각의 헤드로 나누기
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        # 스케일드 닷 프로덕트 어텐션 수행
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        # 헤드 결합
        out = self.combine_heads(out)
        # 최종 선형 변환 수행
        out = self.linear(out)
            
        return out, attention_weights


In [20]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        # 첫 번째 fully-connected layer (linear layer) - 활성화 함수는 ReLU
        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        # 두 번째 fully-connected layer (linear layer)
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        # 주어진 입력을 첫 번째 fully-connected layer에 전달하고 ReLU 활성화 함수를 적용
        out = self.fc1(x)
        # 결과를 두 번째 fully-connected layer에 전달
        out = self.fc2(out)
            
        return out

In [21]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        # 멀티헤드 어텐션 레이어
        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        # 위치별 피드포워드 네트워크 레이어
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        # 각 레이어마다의 레이어 정규화 레이어
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # 드롭아웃 레이어
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        # Residual Connection을 위한 원본 입력 저장
        residual = x
        # 첫 번째 레이어 정규화
        out = self.norm_1(x)
        # 멀티헤드 어텐션 레이어 수행
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        # 드롭아웃 적용
        out = self.do(out)
        # Residual Connection 수행
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        # Residual Connection을 위한 원본 입력 저장
        residual = out
        # 두 번째 레이어 정규화
        out = self.norm_2(out)
        # 위치별 피드포워드 네트워크 레이어 수행
        out = self.ffn(out)
        # 드롭아웃 적용
        out = self.do(out)
        # Residual Connection 수행
        out += residual
        
        return out, enc_attn

In [22]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        # 디코더 자체 어텐션 레이어
        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        # 디코더-인코더 어텐션 레이어
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        # 위치별 피드포워드 네트워크 레이어
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        # 각 레이어마다의 레이어 정규화 레이어
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # 드롭아웃 레이어
        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        # Residual Connection을 위한 원본 입력 저장
        residual = x
        # 첫 번째 레이어 정규화
        out = self.norm_1(x)
        # 디코더 자체 어텐션 레이어 수행
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        # 드롭아웃 적용
        out = self.do(out)
        # Residual Connection 수행
        out += residual

        '''
        Multi-Head Attention
        '''
        # Residual Connection을 위한 원본 입력 저장
        residual = out
        # 두 번째 레이어 정규화
        out = self.norm_2(out)
        # 디코더-인코더 어텐션 레이어 수행
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        # 드롭아웃 적용
        out = self.do(out)
        # Residual Connection 수행
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        # Residual Connection을 위한 원본 입력 저장
        residual = out
        # 세 번째 레이어 정규화
        out = self.norm_3(out)
        # 위치별 피드포워드 네트워크 레이어 수행
        out = self.ffn(out)
        # 드롭아웃 적용
        out = self.do(out)
        # Residual Connection 수행
        out += residual

        return out, dec_attn, dec_enc_attn

In [23]:
class Encoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        
        # 인코더 레이어를 n_layers 개수만큼 생성
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                           for _ in range(n_layers)]
        
        # 드롭아웃 레이어
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        # 입력 데이터 초기화
        out = x
    
        # 각 인코더 레이어를 순회하면서 호출
        enc_attns = list()
        for i in range(self.n_layers):
            # 인코더 레이어 호출
            out, enc_attn = self.enc_layers[i](out, mask)
            # 어텐션 가중치 저장
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [24]:
class Decoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Decoder, self).__init__()
        
        # 디코더 레이어를 n_layers 개수만큼 생성
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                           for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        # 입력 데이터 초기화
        out = x
    
        # 각 디코더 레이어를 순회하면서 호출
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            # 디코더 레이어 호출
            out, dec_attn, dec_enc_attn = \
                self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            # 각 레이어에서 얻은 어텐션 가중치 저장
            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        # 최종 디코더 레이어의 출력과 각 레이어에서의 어텐션 가중치 리스트 반환
        return out, dec_attns, dec_enc_attns

In [25]:
class Transformer(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, src_vocab_size, tgt_vocab_size, pos_len, dropout=0.2, shared_fc=True, shared_emb=False):
        super(Transformer, self).__init__()

        # 입력으로 받은 d_model을 float32 타입으로 변경
        self.d_model = tf.cast(d_model, tf.float32)

        # 공유 임베딩 여부에 따라 인코더와 디코더의 임베딩 레이어 생성
        if shared_emb:
            self.enc_emb = self.dec_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        # 위치 인코딩 레이어 생성
        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        # 인코더와 디코더 생성
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        # 출력 레이어 생성
        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        # 공유 출력 레이어 여부에 따라 가중치 공유
        self.shared_fc = shared_fc
        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        # 입력에 임베딩 레이어 적용
        out = emb(x)

        # 공유된 출력 레이어를 사용할 경우 sqrt(d_model)로 스케일 조정
        if self.shared_fc:
            out *= tf.math.sqrt(self.d_model)

        # 위치 인코딩 추가
        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        # 인코더 입력과 디코더 입력에 임베딩 적용
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        # 인코더와 디코더 순전파
        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        dec_out, dec_attns, dec_enc_attns = self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)

        # 출력 레이어를 통과한 최종 로짓 반환
        logits = self.fc(dec_out)

        # 로짓, 인코더 어텐션, 디코더 어텐션, 디코더-인코더 어텐션 반환
        return logits, enc_attns, dec_attns, dec_enc_attns

In [26]:
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

d_model = 512

In [27]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        # 모델의 임베딩 차원과 워머핑 스텝 수 초기화
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        # 학습률을 조절하기 위한 함수 정의
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        # 두 가지 학습률 조절 방법 중 작은 값 선택
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [28]:
# LearningRateScheduler를 사용하여 학습률 스케줄링 설정
learning_rate = LearningRateScheduler(d_model)

# Adam 옵티마이저 설정
optimizer = tf.keras.optimizers.Adam(
    learning_rate,      # 학습률은 LearningRateScheduler에 의해 동적으로 결정됨
    beta_1=0.9,          # Adam 옵티마이저의 하이퍼파라미터 beta_1 설정
    beta_2=0.98,         # Adam 옵티마이저의 하이퍼파라미터 beta_2 설정
    epsilon=1e-9         # Adam 옵티마이저의 하이퍼파라미터 epsilon 설정
)

In [29]:
# SparseCategoricalCrossentropy 손실 함수 객체 생성
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,     # 모델 출력이 로짓인 경우 True, 확률 분포인 경우 False
    reduction='none'      # 손실을 각 샘플에 대해 계산하고, 각 샘플의 손실을 모두 더하지 않음
)

def loss_function(real, pred):
    # 패딩된 부분을 제외하고 유효한 부분만 마스킹
    mask = tf.math.logical_not(tf.math.equal(real, 0))

    # 모델 예측과 실제 값 간의 손실 계산
    loss_ = loss_object(real, pred)

    # 패딩 부분에 대한 마스킹 적용
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    # 마스킹된 부분의 손실을 총합하고, 마스킹의 총합으로 나눠 정규화
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [30]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    '''
    src: 인코더 입력
    tgt: 디코더 입력
    model: 트랜스포머 모델
    optimizer: 사용할 최적화 알고리즘
    '''
    # Decoder의 입력과 최종 타겟 생성
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    # 마스크 생성
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    # 그래디언트 계산을 위한 GradientTape 사용
    with tf.GradientTape() as tape:
        # 모델 순전파 수행
        predictions, enc_attns, dec_attns, dec_enc_attns = model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)

        # 손실 함수 계산
        loss = loss_function(gold, predictions)

    # 그래디언트 계산 및 모델 파라미터 업데이트
    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # 손실과 어텐션 가중치 반환
    return loss, enc_attns, dec_attns, dec_enc_attns

In [31]:
def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [32]:
def translate_answer(sentence, model, src_tokenizer, tgt_tokenizer):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, src_tokenizer, tgt_tokenizer)
    
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [16]:
BATCH_SIZE = 32
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).batch(batch_size=BATCH_SIZE)

In [ ]:
from tqdm.notebook import tqdm

EPOCHS = 3

example = ["지루하다, 놀러가고 싶어.", "오늘 일찍 일어났더니 피곤하다.",
                 "간만에 여자친구랑 데이트 하기로 했어.", "집에 있는다는 소리야."]

for epoch in range(EPOCHS):
    total_loss = 0
    
    # tqdm을 사용하여 진행 상황을 시각적으로 확인
    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count, desc=f'Epoch {epoch + 1}')

    # 훈련 데이터셋의 각 배치에 대해 훈련 수행
    for (batch, (src, tgt)) in enumerate(train_dataset):
        # 훈련 스텝 수행
        loss, _, _, _ = train_step(enc_train, dec_train, transformer, optimizer)
        total_loss += loss

        # tqdm 업데이트
        tqdm_bar.update(1)
        tqdm_bar.set_postfix({'loss': total_loss.numpy() / (batch + 1)})
    
    for example in examples:
        translate_answer(example, transformer, tokenizer, tokenitokenizerzertokenizer)

    # epoch 종료 후 평균 손실 출력
    avg_loss = total_loss / dataset_count
    print(f'Epoch {epoch + 1}, Loss: {avg_loss.numpy()}')
    
    tqdm_bar.close()

Epoch 1:   0%|          | 0/182 [00:00<?, ?it/s]

# 성능 측정하기

In [ ]:
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

In [ ]:
def translate(tokens, model, src_tokenizer, tgt_tokenizer):
    # 입력 토큰을 패딩하여 모델에 전달
    padded_tokens = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=MAX_LEN,
                                                           padding='post')

    # 디코더 입력 초기화
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)

    # 디코딩된 토큰을 저장할 리스트 초기화
    ids = []

    # 최대 길이(MAX_LEN)만큼 반복하여 번역 수행
    for i in range(MAX_LEN):
        # 마스크 생성
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(padded_tokens, output)

        # 모델에 입력하여 다음 토큰 예측
        predictions, _, _, _ = model(padded_tokens, 
                                      output,
                                      enc_padding_mask,
                                      combined_mask,
                                      dec_padding_mask)

        # 예측된 확률 분포에서 가장 높은 확률을 가진 토큰 선택
        predicted_id = tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        # 종료 토큰이 예측되면 번역 결과 반환
        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)  
            return result

        # 예측된 토큰을 결과에 추가
        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    # 최대 길이까지 종료 토큰이 나오지 않으면 현재까지 예측된 토큰 반환
    result = tgt_tokenizer.decode_ids(ids)  
    return result

In [ ]:
def eval_bleu_single(model, src_sentence, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    # 소스 문장 및 타겟 문장을 토큰화하여 인코딩
    src_tokens = src_tokenizer.encode_as_ids(src_sentence)
    tgt_tokens = tgt_tokenizer.encode_as_ids(tgt_sentence)

    # 최대 길이를 초과하는 경우 평가하지 않음
    if (len(src_tokens) > MAX_LEN) or (len(tgt_tokens) > MAX_LEN):
        return None

    # 타겟 문장을 기준으로 BLEU 점수 계산
    reference = tgt_sentence.split()
    candidate = translate(src_tokens, model, src_tokenizer, tgt_tokenizer).split()

    # BLEU 점수 계산 및 출력
    score = sentence_bleu([reference], candidate,
                          smoothing_function=SmoothingFunction().method1)

    if verbose:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", candidate)
        print("Real: ", reference)
        print("Score: %lf\n" % score)
        
    return score

In [ ]:
test_idx = 0  # 테스트 인덱스

# 테스트 인덱스의 영어 문장과 스페인어 실제 번역 문장을 사용하여 BLEU 점수 평가
eval_bleu_single(transformer, 
                 enc_train[test_idx], 
                 dec_train[test_idx], 
                 tokenizer, 
                 tokenizer)

# 회고

- 여전히 갈 길은 멀고, 알다가도 모르겠는 알쏭달쏭함이 있다.
- 시간을 들여서 익숙해질 필요가 있는 것 같다
- 이론은 알겠는데, 코드로 구현하는 게 쉽지가 않은 것 같다.
- 커널은 왜 자꾸 죽을까... 
- 학습과정에서 커널이 죽어서 아쉽지만 여기까지 온 것만으로도 감사하다. 
- 다음 프로젝트때는 더 나은 결과가 있기를! 